In [1]:
import pandas as pd
df = pd.read_csv('https://github.com/codebasics/py/raw/master/ML/14_naive_bayes/titanic.csv')
df.head()

,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
4,5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0


In [2]:
# dropping the useless variable
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0


In [3]:
target = df.Survived                              # dependent variable
inputs = df.drop('Survived',axis='columns')       # independent variables

In [4]:
# you can see that sex column is a text and we want to convert it into a dummy columns because we all know that the machine
# learining model cant handle text 
dummies = pd.get_dummies(inputs.Sex)
dummies.head()

,female,male
0,0,1
1,1,0
2,1,0
3,1,0
4,0,1


In [5]:
# now we are going to append these dummies columns to our inputs dataframe
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head()

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,0,1
1,1,female,38.0,71.2833,1,0
2,3,female,26.0,7.9250,1,0
3,1,female,35.0,53.1000,1,0
4,3,male,35.0,8.0500,0,1


In [6]:
# now we also need to drop the original Sex column because we already have dummies column
inputs.drop('Sex',axis='columns',inplace=True)
inputs.head()

,Pclass,Age,Fare,female,male
0,3,22.0,7.2500,0,1
1,1,38.0,71.2833,1,0
2,3,26.0,7.9250,1,0
3,1,35.0,53.1000,1,0
4,3,35.0,8.0500,0,1


In [7]:
# now want to know if there is na values
inputs.columns[inputs.isna().any()]

# here our Age columns has some na values

Index(['Age'], dtype='object')

In [8]:
inputs.Age[:10] # you can see that there is NaN values

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

In [9]:
# now we are going fill the NaN value with mean
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.head()

,Pclass,Age,Fare,female,male
0,3,22.0,7.2500,0,1
1,1,38.0,71.2833,1,0
2,3,26.0,7.9250,1,0
3,1,35.0,53.1000,1,0
4,3,35.0,8.0500,0,1


In [12]:
# now we are going to split our dataset into train and test sample this is what we usually do before training our model so 
# that the model is not bias when we are trying to calculate this score
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(inputs,target,test_size=0.2)

In [13]:
len(x_train)

712

In [14]:
len(x_test)

179

In [15]:
len(inputs)

891

In [16]:
# now there of couple of naive bayes classes but we are going to use Gaussian naive bayes. Gaussian naive bayes is somethin 
# that we use when you data distribution is normal
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [17]:
# train the model
model.fit(x_train, y_train)

GaussianNB()

In [18]:
model.score(x_test, y_test)

0.7932960893854749

In [19]:
# lets look at x_test samples
x_test[:10]

,Pclass,Age,Fare,female,male
731,3,11.000000,18.7875,0,1
252,1,62.000000,26.5500,0,1
840,3,20.000000,7.9250,0,1
254,3,41.000000,20.2125,1,0
455,3,29.000000,7.8958,0,1
583,1,36.000000,40.1250,0,1
459,3,29.699118,7.7500,0,1
329,1,16.000000,57.9792,1,0
882,3,22.000000,10.5167,1,0
827,2,1.000000,37.0042,0,1


In [20]:
y_test[:10]

731    0
252    0
840    0
254    0
455    1
583    0
459    0
329    1
882    0
827    1
Name: Survived, dtype: int64

In [21]:
model.predict(x_test[:10])

array([0, 0, 0, 1, 0, 0, 0, 1, 1, 0], dtype=int64)

In [22]:
# you can also use predict probability function to figure out what are the probability of each class
model.predict_proba(x_test[:10])

# here 1st 0.98 probability of person didnt survive and 0.01 probability he\she survived.

array([[0.98801912, 0.01198088],
       [0.83533791, 0.16466209],
       [0.99071682, 0.00928318],
       [0.13276014, 0.86723986],
       [0.99189193, 0.00810807],
       [0.8906018 , 0.1093982 ],
       [0.99192147, 0.00807853],
       [0.00448159, 0.99551841],
       [0.12643085, 0.87356915],
       [0.94335323, 0.05664677]])